# Controllable loop process
> iteration with callbacks

In [1]:
# default_exp loop

In [2]:
# export 
import numpy as np
import pandas as pd
import math

In [3]:
from time import sleep

In [4]:
# export
class Stuff:
    _is_stuff = True
    def __init__(self,name):
        self.name = name
        self.cases = dict()
        self.funcs = dict()
        
    def __setitem__(self,k,v):
        setattr(self,k,v)
        
    def __setattr__(self,k,v):
        if k not in ["name","cases","funcs"]:
            self.cases[k] = v
        else:
            super().__setattr__(k,v)
        
    def __getitem__(self,k):
        return self.cases[k]
    
    def append(self,obj):
        self[f"{self.name}_{len(self+1)}"] = obj
        
    def clear(self):
        self.cases = dict()
    
    def __getattr__(self,k):
        if k in self.cases:
            return self.cases[k]
        elif k in self.funcs:
            return self.funcs[k]
        else:
            return super().__getattr__()
        
    def __repr__(self,):
        return f"🍄:{self.name}[\n\t"+"\n\t".join(self.cases.keys())+"\n]"
    
    def __len__(self):
        return len(self.cases)
    
    def apply(self,func,scope = None):
        """
        func, the function to apply on each element
        scope, list, name the scope of element
        """
        def apply_func(*args,**kwargs):
            for k,case in self.cases.items():
                if scope!=None:
                    if k not in scope:
                        continue
                if type(func)==str:
                    self[k] = getattr(case,func)(*args,**kwargs)
                else:
                    self[k] = func(case,*args,**kwargs)
        return apply_func
    
    def update(self,dictionary):
        for k, v in dictionary.items():
            self[k]=v
    
    def __call__(self,func=None):
        """
        func: str or callable, 
            if str, it will run on each subject case's such named function
            if callable, it will call on the callable, with the 1st arg being the subject case
        args: extra args
        kwargs: extra key word arguments
        
        return: a list of returned results
        """
        def run_funcs(*args,**kwargs):
            results = dict()
            for k,case in self.cases.items():
                if type(func)==str:
                    results.update({k:getattr(case,func)(*args,**kwargs)})
                elif func==None:
                    results.update({k:getattr(case,"__call__")(*args,**kwargs)})
                else:
                    results.update({k:func(case,*args,**kwargs)})
            return results
        return run_funcs

In [5]:
words = Stuff("model")
words["G"] = "generative_model"
words["D"] = "discriminative_model"

In [6]:
def to_device(single_model,device):
    print(f"{single_model} put to device:{device}")
    # single_model.to(device)

In [7]:
words(to_device)(2)

generative_model put to device:2
discriminative_model put to device:2


{'G': None, 'D': None}

In [8]:
words("upper")()

{'G': 'GENERATIVE_MODEL', 'D': 'DISCRIMINATIVE_MODEL'}

In [9]:
words("__len__")()

{'G': 16, 'D': 20}

Both of the following syntax works

In [10]:
words.G

'generative_model'

In real cases we can do
```python
model("to")("cuda:1")
```

In [11]:
words(to_device)("cuda:1")

generative_model put to device:cuda:1
discriminative_model put to device:cuda:1


{'G': None, 'D': None}

In [12]:
# export
from tqdm import tqdm
from types import MethodType
import inspect
from functools import partial

def method4all(f):
    """
    Use this function as a decorator,
    The decorated function under Loop class can be used on outer layer
    """
    setattr(f,"forall",True)
    return f

class StorageCore:
    def __init__(self,start_layer):
        self.layers = []
        self.lmap = dict()
        self.forall_pool = dict()
        self.new_layer(start_layer)
        self.i = -1
        self.epoch = -1
    
    def new_layer(self,layer):
        layer.core = self
        self.layers.append(layer)

        self.lmap[layer._loop_layer]=layer
        if hasattr(self,layer.name):
            raise KeyError(f"Layer name already exist: {layer.name}")
        setattr(self,layer.name,layer)
        self.update_forall(layer)
        self.assign_forall(layer)
  
    def __repr__(self):
        return str(self.lmap)
        
    def for_all_functions(self,obj):
        return dict(inspect.getmembers(obj, 
                  predicate=lambda x:hasattr(x,"forall")))
    
    def update_forall(self,obj):
        self.forall_pool.update(self.for_all_functions(obj))
        
    def assign_forall(self,obj):
        for name,f in self.forall_pool.items():
            setattr(obj,name,f)

class Loop(list):
    """
    Basic loop class
    """
    _loop_layer = 0.
    _loop_obj = True
    def __init__(self,iterable = [],name=None):
        self.iterable  = iterable
        self.next_layer = None
        self.name = name if name!=None else self.__class__.__name__
        
        if hasattr(iterable,"_loop_layer"):
            self._loop_layer = iterable._loop_layer + 1
            iterable.next_layer = self
            iterable.core.new_layer(self)
        else:
            self.core = StorageCore(self)
        
    def __call__(self):
        pass
    
    def __repr__(self):
        return f"layer🍰{self.name}"
    
    def on(self,iterable):
        self.iterable = iterable
        self._loop_layer = iterable._loop_layer + 1
        
    def func_detail(self,f):
        detail = dict({"🐍func_name":f.__name__,
                       "⛰doc":f.__doc__,
                       "😝var":",".join(f.__code__.co_varnames),
                       "😜names":",".join(f.__code__.co_names),
                      })
        return detail
    
    def func_detail_df(self):
        funcs = []
        result = []
        for idx,layer in self.core.lmap.items():
            for fname,f in self.core.for_all_functions(layer).items():
                if id(f) not in funcs:
                    dt = self.func_detail(f)
                    dt.update(layer=str(layer))
                    result.append(dt)
                    funcs.append(id(f))
        return pd.DataFrame(result)
    
    def search(self):
        from forgebox.widgets import search_box
        pd.set_option('display.max_colwidth', 0)
        doc_df = self.func_detail_df()
        doc_df["⛰doc"]= doc_df["⛰doc"].apply(lambda x:str(x).replace("\n"," "))
        search_box(doc_df, ["🐍func_name","⛰doc","layer","😝var","😜names"])
    
    def summary(self):
        rt = f"Loop layer {self.name} summary:\n"
        rt+= "="*50+"\n"
        funcs = []
        for idx,layer in self.core.lmap.items():
            rt+= f"🍰layer{idx}\t{str(layer)}\n"
            for fname,f in self.core.for_all_functions(layer).items():
                if id(f) not in funcs:
                    rt+="\t"
                    rt+="\n\t".join(f"[{k}]\t{v}" for k,v in self.func_detail(f).items())
                    rt+="\n\t"+"."*35+"\n"
                    funcs.append(id(f))
            rt+="-"*50+"\n"
        rt+= "="*50+"\n"
        print(rt)
    
    def __len__(self):
        return self.iterable.__len__()
    
    def run(self,run_nb=1):
        """
        Run through iteration
        run start_call for every layer on the start of iteration
            run __call__ for every layer for each iteration
        run end_call for every layer on the end of iteration
        """
        self.before_1st_callon()
        for i in range(run_nb):
            self.one_epoch()
        self.after_last_callon()
        
    def one_epoch(self):
        first = self.layers[0]
        self.refresh_i()
        first.start_callon()
        for element in first:
            first()
        first.end_callon()
        
    def __getitem__(self,idx):
        return self.iterable[idx]
    
    def __setitem__(self,idx,v):
        self.iterable[idx] = v
        
    def refresh_i(self):
        self.core.i=-1
        self.core.epoch+=1
        
    def update_i(self):
        self.core.i+=1

    def downstream_wrap(self,f):return f()
    def downstream_start_wrap(self,f):return f()
    def downstream_end_wrap(self,f):return f()
    def downstream_before_1st_wrap(self,f): return f()
    def downstream_after_last_wrap(self,f): return f()
    
    def downstream(self,f):
        """
        set downstream wrapper on callback, 
        The callback will happend in the local realm of the deocrated function
        example
        @loop.downstream
        def control_layer(self,callback):
            try:
                callback()
            except:
                print("error happened")
        """
        setattr(self,"downstream_wrap",MethodType(f,self))
        return f
    
    def downstream_start(self,f):
        """
        set downstream wrapper on start callback, 
        The start_callback will happend in the local realm of the deocrated function
        example
        @loop.downstream_start
        def control_layer(self,start_callback):
            try:
                start_callback()
            except:
                print("error happened")
        """
        setattr(self,"downstream_start_wrap",MethodType(f,self))
        return f
    
    def downstream_end(self,f):
        """
        set downstream wrapper on end callback, 
        The end_callback will happend in the local realm of the deocrated function
        @loop.downstream_end
        def control_layer(self,end_callback):
            try:
                end_callback()
            except:
                print("error happened")
        """
        setattr(self,"downstream_end_wrap",MethodType(f,self))
        return f
    
    def callon(self):
        self()
        self.downstream_wrap(self.iter_cb)
            
    def start_callon(self):
        self.start_call()
        self.downstream_start_wrap(self.start_cb)
            
    def end_callon(self):
        self.end_call()
        self.downstream_end_wrap(self.end_cb)
        
    def before_1st_callon(self):
        self.before_1st_call()
        self.downstream_before_1st_wrap(self.before_1st_cb)
        
    def after_last_callon(self):
        self.after_last_call()
        self.downstream_before_1st_wrap(self.after_last_cb)
            
    def iter_cb(self):
        """
        call back during each iteration
        """
        if self.next_layer!=None:
            self.next_layer.callon()
        self.after()
            
    def after(self):
        pass
            
    def start_cb(self):
        """
        callback at the start of iteration
        """
        if self.next_layer!=None:
            self.next_layer.start_callon()
    
    def end_cb(self):
        """
        callback at the end of iteration
        """
        if self.next_layer!=None:
            self.next_layer.end_callon()
            
    def before_1st_cb(self):
        """
        callback before the 1st of iteration
        """
        if self.next_layer!=None:
            self.next_layer.before_1st_callon()
            
    def after_last_cb(self):
        """
        callback after the last of iteration
        """
        if self.next_layer!=None:
            self.next_layer.after_last_callon()
            
    def before_1st_call(self):pass
    def after_last_call(self):pass
    def start_call(self):pass
    def end_call(self):pass
            
    def __iter__(self,):
        for element in self.iterable:
            if self._loop_layer ==0:
                self.update_i()
            self.core.element = element
            self.callon()
            yield self.element
            
    def __getattr__(self,k):
        return getattr(self.core,k)
    
    def is_newloop(self):
        """
        return Bool:Is this a new loop ready to start
        """
        return (self.i==-1 or self.i==self.__len__()-1)

## Loop-in-loop concept
Loop and its inheriting descendants runs on iterable, incuding other ```Loop```

In [13]:
l = Loop(range(5,10),name = "hahahha")
l

layer🍰hahahha

In [14]:
class Printer(Loop):
    def __call__(self):
        print(f"i_{self.core.i}",end="\t")
        print(self.core.element)
        
    def start_call(self):
        print(f"start of epoch {self.epoch}")
        
    def end_call(self):
        print(f"end of epoch {self.epoch}")
l2 = Printer(l)

In [15]:
l2.run()

start of epoch 0
i_0	5
i_1	6
i_2	7
i_3	8
i_4	9
end of epoch 0


## Progress bar

In [16]:
# export
class ProgressBar(Loop):
    def __init__(self,iterable=[],jupyter = True,mininterval = 1.,leave=False):
        super().__init__(iterable,"ProgressBar")
        """
        jupyter:bool, using the jupyter progressbar (or the console pb)?
            default True
        mininterval: minimun update interval
        leave:bool, if true, we do not remove the progress bar at the end
            default: False
        """
        
        if jupyter: # jupyter widget
            from tqdm.notebook import tqdm
            
        else: # compatible for console print
            from tqdm import tqdm
            
        self.tqdm = tqdm
        self.leave = leave
        self.mininterval = mininterval
        self.data = dict()
        
    @method4all
    def pgbar_data(self,data):
        """
        update progress bar with python dictionary
        data: python dictionary
        """
        self.t.set_postfix(data)
    
    @method4all
    def pgbar_description(self,text):
        """
        update progress bar prefix with text string
        """
        self.t.set_description_str(f"{text}")
        
    def start_call(self):
        self.create_bar()
        
    def end_call(self):
        self.t.close()
        
    def __call__(self):
        self.t.update(1)
            
    def create_bar(self):
        self.t = self.tqdm(total=len(self.iterable),leave=self.leave,
                           mininterval=self.mininterval)

In [17]:
from tqdm.notebook import tqdm,tqdm_notebook

In [18]:
pb = ProgressBar(Loop(range(5,10)),jupyter = True,leave=True)
pb.run()

### Downstream wraper
> A unique way to contain downstream task in a function, hence a good Error Handling mechanism

In cases you don't want error stop the iteration, meanwhile taking down all the error message.

In [19]:
# export
def error_tolerate(self,downstream_func):
    """
    downstream_func:Downstream function
    """
    try:
        downstream_func()
    except Exception as e:
        self.errors.append(dict(stage=downstream_func.__name__,
                                i=self.i,
                                epoch=self.epoch,
                                error=e))
    
class Tolerate(Loop):
    """
    Tolerate any error happened downstream
    layer2 = Tolerate(upstream_iterable)
    # build downstream tasks
    layer3 = OtherApplication(layer2)
    layer3.run()
    # show the happened error message
    layer3.error_list()
    """
    def __init__(self,iterable = []):
        super().__init__(iterable,)
        self.errors = list()
        # wrap downstream task with error tolerate
        for decorator in [self.downstream,
                          self.downstream_start,
                          self.downstream_end]:
            decorator(error_tolerate)
    
    @method4all
    def error_list(self):
        """
        A list of errors happend so far
        """
        return self.errors
    
    def end_call(self,):
        le = len(self.error_list())
        if le>0:
            print(f"WARNING:{le} errors")

class LambdaCall(Loop):
    def __init__(self,iterable = [],
                 func = lambda x:x,
                 start_func = lambda x:x,
                 end_func = lambda x:x
                ):
        super().__init__(iterable,name=f"Lambda<{hex(id(self))}>")
        self.func = func
        self.start_func = end_func
        self.end_func = end_func
    
    def __call__(self):
        self.func(self)
        
    def start_call(self):
        self.start_func(self)
    
    def end_call(self):
        self.end_func(self)

In [20]:
l1 = ProgressBar(Loop(range(1,10)))
error_loop = LambdaCall(l1,func= lambda x:(5-x.element)**(-2))

Running this will yield error

```python
error_loop.run()
```

In [21]:
def error_on_purpose(x):
    if x.element>5:
        raise ValueError(f"value too small, {x.element}")

l1 = ProgressBar(Loop(range(1,10)))
# add the tolerate layer in between
l2 = Tolerate(l1)
# tolerate can only tolerate the erorr on downstream
tolerated_loop = LambdaCall(l2,func=error_on_purpose,end_func=error_on_purpose)
l2.run()

you can use ```error_list``` function to print all the happened error

In [22]:
import pandas as pd
pd.DataFrame(l2.error_list())

,stage,i,epoch,error
0,start_cb,-1,0,'StorageCore' object has no attribute 'element'
1,iter_cb,5,0,"value too small, 6"
2,iter_cb,6,0,"value too small, 7"
3,iter_cb,7,0,"value too small, 8"
4,iter_cb,8,0,"value too small, 9"
5,end_cb,8,0,"value too small, 9"


In [23]:
l2.summary()

Loop layer Tolerate summary:
🍰layer0.0	layer🍰Loop
--------------------------------------------------
🍰layer1.0	layer🍰ProgressBar
	[🐍func_name]	pgbar_data
	[⛰doc]	
        update progress bar with python dictionary
        data: python dictionary
        
	[😝var]	self,data
	[😜names]	t,set_postfix
	...................................
	[🐍func_name]	pgbar_description
	[⛰doc]	
        update progress bar prefix with text string
        
	[😝var]	self,text
	[😜names]	t,set_description_str
	...................................
--------------------------------------------------
🍰layer2.0	layer🍰Tolerate
	[🐍func_name]	error_list
	[⛰doc]	
        A list of errors happend so far
        
	[😝var]	self
	[😜names]	errors
	...................................
--------------------------------------------------
🍰layer3.0	layer🍰Lambda<0x119422e30>
--------------------------------------------------



You can search for the function documentation

In [24]:
l2.search()

interactive(children=(Text(value='', description='KeyWord'), Output()), _dom_classes=('widget-interact',))

### Application upon progress bar
The function ```pgbar_data``` is decorated under ```method4all```, hence the function can be used else where

In [25]:
from datetime import datetime
class Application(Loop):
    def __call__(self):
        # create some data
        loop_data = dict(epoch=self.epoch,i = self.i,
                         val = self.element,)
        # update data to progress bar
        self.pgbar_data(loop_data)
        self.pgbar_description(str(datetime.now().strftime("%H:%M:%S")))

l3 = Application(ProgressBar(Loop(range(5,10)),jupyter=True,leave=True))
l3.run()

## Print out the summary of the loop

In [26]:
l3.summary()

Loop layer Application summary:
🍰layer0.0	layer🍰Loop
--------------------------------------------------
🍰layer1.0	layer🍰ProgressBar
	[🐍func_name]	pgbar_data
	[⛰doc]	
        update progress bar with python dictionary
        data: python dictionary
        
	[😝var]	self,data
	[😜names]	t,set_postfix
	...................................
	[🐍func_name]	pgbar_description
	[⛰doc]	
        update progress bar prefix with text string
        
	[😝var]	self,text
	[😜names]	t,set_description_str
	...................................
--------------------------------------------------
🍰layer2.0	layer🍰Application
--------------------------------------------------



In [27]:
l3.run()

## Event

Event allows add-hoc callbacks to be created

In [28]:
# export 
    
class Event(Loop):
    """
    An event is the landmark with in 1 iteration
    """
    def __init__(self,iterable=[]):
        event_name = self.__class__.__name__
        super().__init__(iterable,event_name)
        self.event_name = event_name
        self.cbs = []
        self.create_cb_deco("on")
        self.create_cb_deco("before_1st")
        self.create_cb_deco("every_start")
        self.create_cb_deco("every_end")
        self.create_cb_deco("after_last")
        self.core.update_forall(self)
        self.core.assign_forall(self)
        
    def __repr__(self):
        return f"event🌏{self.event_name}"
    
    def create_cb_deco(self,moment):
        event_name = self.event_name
        def wraper(cls,f):return getattr(self,moment)(f)
        wraper.__name__ = f"{moment}_{event_name}"
        wraper.__doc__ = f"""
            Append new {moment} callback for event:{event_name}
            Use this function as decorator
        """
        setattr(self,wraper.__name__,MethodType(method4all(wraper),self))
        
    def __call__(self):
        if len(self.cbs)>0:
            self.cbs[0].callon()
            
    def start_call(self):
        if len(self.cbs)>0:
            self.cbs[0].start_callon()
    
    def end_call(self):
        if len(self.cbs)>0:
            self.cbs[0].end_callon()
    
    def on(self,f):
        class EventCB(Loop):
            def __init__(self_,iterable=[]):
                super().__init__(iterable=iterable,
                                 name = f"ON_{self.event_name}_{self.cbs.__len__()+1}_{f.__name__}")
                self_.f = f
                
            def __call__(self_): self_.f(self)
                
        new_cb = EventCB()
        self.new_event_cb(new_cb)
        return f

    def every_start(self,f):
        class EventCbEveryStart(Loop):
            def __init__(self_,iterable=[]):
                super().__init__(iterable=iterable,
                                 name = f"EVERY_START_{self.event_name}_{self.cbs.__len__()+1}_{f.__name__}")
                self_.f = f
                
            def start_call(self_): self_.f(self)
                
        new_cb = EventCbEveryStart()
        self.new_event_cb(new_cb)
        return f
    
    def before_1st(self,f):
        class EventCbBefore1st(Loop):
            def __init__(self_,iterable=[]):
                super().__init__(iterable=iterable,
                                 name = f"BEFORE_1ST_{self.event_name}_{self.cbs.__len__()+1}_{f.__name__}")
                self_.f = f
                
            def before_1st_call(self_): self_.f(self)
                
        new_cb = EventCbBefore1st()
        self.new_event_cb(new_cb)
        return f
    
    def every_end(self,f):
        class EventCbEveryEnd(Loop):
            def __init__(self_,iterable=[]):
                super().__init__(iterable=iterable,
                                 name = f"EVERY_END_{self.event_name}_{self.cbs.__len__()+1}_{f.__name__}")
                self_.f = f
                
            def end_call(self_): self_.f(self)
                
        new_cb = EventCbEveryEnd()
        self.new_event_cb(new_cb)
        return f
    
    def after_last(self,f):
        class EventCbAfter(Loop):
            def __init__(self_,iterable=[]):
                super().__init__(iterable=iterable,
                                 name = f"AFTER_LAST_{self.event_name}_{self.cbs.__len__()+1}_{f.__name__}")
                self_.f = f
                
            def after_last_call(self_): self_.f(self)
                
        new_cb = EventCbAfter()
        self.new_event_cb(new_cb)
        return f
                
    def new_event_cb(self,new_cb):
        new_cb.core.update_forall(new_cb)
        new_cb.core.assign_forall(new_cb)
        new_cb.core = self.core
        if len(self.cbs)>0:
            self.cbs[-1].next_layer = new_cb
#             self.cbs[-1].new_layer(new_cb)
        self.cbs.append(new_cb)

In [29]:
class ToString(Loop):
    def __call__(self,):
        self.core.element = f"variable_{self.i}"

class ChangeString(Event):
    def __init__(self,iterable=[]):
        super().__init__(iterable=iterable)

In [30]:
def get_frame(li):
    layer1 = ProgressBar(Loop(li,"base"),leave=True)
    layer2 = ToString(layer1,)
    layer3 = ChangeString(layer2)
    # you can use this "on" syntax
    @layer3.ChangeString.on
    def trim(self):
        self.core.element = f"Var_{self.element}"
        
    # or this syntax
    @layer3.on_ChangeString
    def update(self):
        self.pgbar_description(self.element)
        
    @layer3.before_1st_ChangeString
    def print_start(self):
        """
        decorated
        """
        print(f"start_{self.epoch}_{self.i}")
        print(self.name)
    return layer3

In [31]:
l = get_frame(range(10))

In [32]:
l.core

{0.0: layer🍰base, 1.0: layer🍰ProgressBar, 2.0: layer🍰ToString, 3.0: event🌏ChangeString}

In [33]:
l.cbs

[layer🍰ON_ChangeString_1_trim,
 layer🍰ON_ChangeString_2_update,
 layer🍰BEFORE_1ST_ChangeString_3_print_start]

In [34]:
l.summary()

Loop layer ChangeString summary:
🍰layer0.0	layer🍰base
--------------------------------------------------
🍰layer1.0	layer🍰ProgressBar
	[🐍func_name]	pgbar_data
	[⛰doc]	
        update progress bar with python dictionary
        data: python dictionary
        
	[😝var]	self,data
	[😜names]	t,set_postfix
	...................................
	[🐍func_name]	pgbar_description
	[⛰doc]	
        update progress bar prefix with text string
        
	[😝var]	self,text
	[😜names]	t,set_description_str
	...................................
--------------------------------------------------
🍰layer2.0	layer🍰ToString
--------------------------------------------------
🍰layer3.0	event🌏ChangeString
	[🐍func_name]	after_last_ChangeString
	[⛰doc]	
            Append new after_last callback for event:ChangeString
            Use this function as decorator
        
	[😝var]	cls,f
	[😜names]	getattr
	...................................
	[🐍func_name]	before_1st_ChangeString
	[⛰doc]	
            Append new before_1st ca

In [35]:
l.layers

[layer🍰base, layer🍰ProgressBar, layer🍰ToString, event🌏ChangeString]

In [36]:
l.run()

In [37]:
layer = l.layers[0]
print(layer)
while layer.next_layer!=None:
    layer=layer.next_layer
    print(layer.start_call.__doc__)

layer🍰base
None
None
None


## Parallel

In [64]:
# export
from joblib import Parallel,delayed

In [41]:
from math import sqrt

In [42]:
def heavy_calc(x):
    sleep(1)
    return sqrt(x)

In [44]:
%%time
list(map(heavy_calc,range(10)))

CPU times: user 933 µs, sys: 1.41 ms, total: 2.35 ms
Wall time: 10 s


[0.0,
 1.0,
 1.4142135623730951,
 1.7320508075688772,
 2.0,
 2.23606797749979,
 2.449489742783178,
 2.6457513110645907,
 2.8284271247461903,
 3.0]

In [46]:
%%time
Parallel(n_jobs=5,)(delayed(heavy_calc)(i) for i in range(10))

CPU times: user 26.5 ms, sys: 21.8 ms, total: 48.3 ms
Wall time: 2.79 s


[0.0,
 1.0,
 1.4142135623730951,
 1.7320508075688772,
 2.0,
 2.23606797749979,
 2.449489742783178,
 2.6457513110645907,
 2.8284271247461903,
 3.0]

In [89]:
def showstuff(*args):
    print(args)
    for i in args[0]:
        print(i)
showstuff(a for a in range(5))

(<generator object <genexpr> at 0x11b3761d0>,)
0
1
2
3
4


In [123]:
class MultiProc:
    def __init__(self,loop,n_jobs = 6):
        self.loop = loop
        self.n_jobs = n_jobs
        self.parallel = Parallel(n_jobs=n_jobs)
        
    def __enter__(self):
        loop = self.loop
        def one_epoch_parallel(loop):
            first = loop.layers[0]
            loop.refresh_i()
            first.start_callon()
            print(1)
            self.parallel(delayed(first.__call__)() for element in first)
            print(2)
            first.end_callon()
        self.old = loop.one_epoch
        setattr(loop,"one_epoch",MethodType(one_epoch_parallel,loop))
        return
    
    def __exit__(self,exc_type,exc_val,exc_tb):
        setattr(self.loop,"one_epoch",self.old)
        return 

In [124]:
class Calc(Loop):
    def __init__(self,iterable):
        super().__init__(iterable=iterable)
        
    def __call__(self):
        sleep(1)
        self.pgbar_data(dict(sqrt=sqrt(self.element)))
        
    def end_call(self):
        print("done")

In [125]:
basic_loop = ProgressBar(Loop(range(10)))
test_li = Calc(basic_loop)

In [126]:
%%time
test_li.run()

done
CPU times: user 55.5 ms, sys: 8.47 ms, total: 64 ms
Wall time: 10.1 s


## Chunkify for batch cutting

In [38]:
# export
def chunkify(*iterables,bs = 32):
    if len(iterables)>1:
        return list(tuple(l[i*bs:(i+1)*bs] for l in iterables) \
                for i in range(math.ceil(len(iterables[0])/bs)))
    else:
        return list(iterables[0][i*bs:(i+1)*bs] \
                for i in range(math.ceil(len(iterables[0])/bs)))

In [39]:
chunkify(list(range(10)),list(range(10,20)),bs=3)

[([0, 1, 2], [10, 11, 12]),
 ([3, 4, 5], [13, 14, 15]),
 ([6, 7, 8], [16, 17, 18]),
 ([9], [19])]